<a href="https://colab.research.google.com/github/fralfaro/MAT281_2024/blob/main/docs/labs/lab_032.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°032


Esta semana revisaremos datos del **Índice de Libertad de Prensa** que confecciona cada año la asociación de Reporteros Sin Fronteras.


## Diccionario de datos


|Variable       |Clase               |Descripción |
|:--------------|:-------------------|:-----------|
| codigo_iso | caracter | Código ISO del país |
| pais | caracter | País |
| anio | entero | Año del resultado |
| indice | entero | Puntaje Índice Libertad de Prensa (menor puntaje = mayor libertad de prensa) |
| ranking | entero | Ranking Libertad de Prensa |


## Fuente original y adaptación
Los datos fueron extraídos de [The World Bank](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019). La fuente original es [Reporteros sin Fronteras](https://www.rsf-es.org/).

Por otro lado, estos archivos han sido modificado intencionalmente para ocupar todo lo aprendido en clases. A continuación, una breve descripción de cada uno de los data frames:

* **libertad_prensa_codigo.csv**: contiene la información codigo_iso/pais. Existe un código que tiene dos valores.
* **libertad_prensa_01.csv**: contiene la información pais/anio/indice/ranking. Los nombres de las columnas estan en mayúscula (antes del año 2010).
* **libertad_prensa_02.csv**: contiene la información pais/anio/indice/ranking. Los nombres de las columnas estan en mayúscula (después del año 2010).




In [159]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281_2024/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281_2024/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281_2024/main/docs/labs/data/libertad_prensa_codigo.csv')

 El objetivo es tratar de obtener la mayor información posible de este conjunto de datos. Para cumplir este objetivo debe resolver las siguientes problemáticas:

1. Lo primero será juntar toda la información en un _solo archivo_, para ello necesitamos seguir los siguientes pasos:

 * a) Crear el archivo **df_anio**, que contenga la información de **libertad_prensa_anio.csv** para cada año. Luego, normalice el nombre de las columnas a minúscula.
 * b) Encuentre y elimine el dato que esta duplicado en el archivo **df_codigo**.
 * c) Crear el archivo **df** que junte la información del archivo **df_anio** con **df_codigo** por la columna _codigo_iso_.

> **Hint**: Para juntar por _anio_ ocupe la función **pd.concat**. Para juntar información por columna ocupe **pd.merge**.

In [160]:
# Paso a)
df_anio1 = pd.read_csv(archivos_anio[0],sep=",")
df_anio2 = pd.read_csv(archivos_anio[1],sep=",")
df_anio=pd.concat([df_anio1,df_anio2])
df_anio.columns = df_anio.columns.str.lower()


df = pd.DataFrame()

In [161]:
#Paso b)
print(df_codigos["pais"].duplicated().sum())
df_codigos = df_codigos.drop_duplicates(subset="pais")
print(df_codigos["pais"].duplicated().sum())

1
0


In [162]:
#Paso c)
df = pd.merge(df_anio,df_codigos, on="codigo_iso")
print(df)

     codigo_iso  anio  indice  ranking         pais
0           AFG  2001   35.50     59.0  Afghanistán
1           AFG  2002   40.17     78.0  Afghanistán
2           AFG  2003   28.25     49.0  Afghanistán
3           AFG  2004   39.17     62.0  Afghanistán
4           AFG  2005   44.25     67.0  Afghanistán
...         ...   ...     ...      ...          ...
3055        ZWE  2017   41.44    128.0         malo
3056        ZWE  2018   40.53    126.0     Zimbabue
3057        ZWE  2018   40.53    126.0         malo
3058        ZWE  2019   42.23    127.0     Zimbabue
3059        ZWE  2019   42.23    127.0         malo

[3060 rows x 5 columns]


2. Encontrar:
   * ¿Cuál es el número de observaciones en el conjunto de datos?   
   * ¿Cuál es el número de columnas en el conjunto de datos?   
   * Imprime el nombre de todas las columnas  
   * ¿Cuál es el tipo de datos de cada columna?
   * Describir el conjunto de datos (**hint**: .describe())
    

In [163]:
# a)
print(f"Hay {df.shape[0]} observaciones")
#b)
print(f"Hay {len(df.columns)}. columnas")
#c)
print("Las columnas son:")
for i in range(len(df.columns)):
  print(df.columns[i])
#d)
print(f"Las columnas tienen tipo de dato:\n {df.dtypes}")
df.describe()


Hay 3060 observaciones
Hay 5. columnas
Las columnas son:
codigo_iso
anio
indice
ranking
pais
Las columnas tienen tipo de dato:
 codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object


,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.844595,478.038773
std,5.786024,2695.521510,6474.928764
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,35.000000
50%,2009.000000,28.000000,71.000000
75%,2015.000000,41.477500,110.000000
max,2019.000000,64536.000000,121056.000000


3. Desarrolle una función `resumen_df(df)` para encontrar el total de elementos distintos y vacíos por columnas.

In [164]:
# respuesta
def resumen_df(df):
    """
    Función para generar un resumen de un DataFrame que incluye
    el número de elementos distintos y vacíos por columna.

    Args:
    df (pd.DataFrame): DataFrame a resumir.

    Returns:
    pd.DataFrame: DataFrame resumen con el nombre de las columnas,
                  cantidad de elementos distintos y cantidad de elementos vacíos.
    """
    # Crear un DataFrame de resultado con los nombres de las columnas
    nombres = df.columns
    result = pd.DataFrame({'nombres': nombres})

    # Calcular el número de elementos distintos por columna
    df_elem_col = df.nunique()
    result['elementos_distintos'] = result["nombres"].map(df_elem_col)
    # Calcular el número de elementos vacíos (NaN) por columna
    result['elementos_vacios'] = result["nombres"].map(df.isna().sum())

    return result

In [165]:
# retornar
resumen_df(df)

,nombres,elementos_distintos,elementos_vacios
0,codigo_iso,179,0
1,anio,17,0
2,indice,1541,396
3,ranking,193,223
4,pais,180,0


4. Para los paises latinoamericano, encuentre por año  el país con mayor y menor `indice`.

 * a) Mediante un ciclo _for_.
 * b) Mediante un  _groupby_.

In [205]:

# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']
#Mediante for
anios = df['anio'].unique()
for a in anios:
  df_filtrado = pd.DataFrame()
  df_filtrado_am = pd.DataFrame()
  dffas = pd.DataFrame()

  df_filtrado = df[df["anio"]== a]
  df_filtrado_am = df_filtrado[df["codigo_iso"].isin(america)]
  dffas = df_filtrado_am.sort_values("indice", ascending = False)
  dffad = df_filtrado_am.sort_values('indice', ascending = True)
  maxp = dffas.iloc[0]['pais']
  maxi = dffas.iloc[0]['indice']
  minp = dffad.iloc[0]['pais']
  mini = dffad.iloc[0]['indice']
  print(f"En el año {a} el pais con mayor indice es: {maxp} con un indice de {maxi}")
  print(f"En el año {a} el pais con menor indice es: {minp} con un indice de {mini}")


En el año 2001 el pais con mayor indice es: Cuba con un indice de 90.3
En el año 2001 el pais con menor indice es: Canadá con un indice de 0.8
En el año 2002 el pais con mayor indice es: Cuba con un indice de 97.83
En el año 2002 el pais con menor indice es: Trinidad y Tobago con un indice de 1.0
En el año 2003 el pais con mayor indice es: Argentina con un indice de 35826.0
En el año 2003 el pais con menor indice es: Trinidad y Tobago con un indice de 2.0
En el año 2004 el pais con mayor indice es: Cuba con un indice de 87.0
En el año 2004 el pais con menor indice es: Trinidad y Tobago con un indice de 2.0
En el año 2005 el pais con mayor indice es: Cuba con un indice de 95.0
En el año 2005 el pais con menor indice es: Bolivia con un indice de 4.5
En el año 2006 el pais con mayor indice es: Cuba con un indice de 96.17
En el año 2006 el pais con menor indice es: Canadá con un indice de 4.88
En el año 2007 el pais con mayor indice es: Cuba con un indice de 88.33
En el año 2007 el pais co

<ipython-input-205-fde08946af09>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtrado_am = df_filtrado[df["codigo_iso"].isin(america)]
<ipython-input-205-fde08946af09>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtrado_am = df_filtrado[df["codigo_iso"].isin(america)]
<ipython-input-205-fde08946af09>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtrado_am = df_filtrado[df["codigo_iso"].isin(america)]
<ipython-input-205-fde08946af09>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtrado_am = df_filtrado[df["codigo_iso"].isin(america)]
<ipython-input-205-fde08946af09>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtrado_am = df_filtrado[df["codigo_iso"].isin(america)]
<ipython-input-205-fde08946af09>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_fi

In [237]:
#mediante groupby
df_filam = df[df['codigo_iso'].isin(america)]
#print(df_filam)
df_america = df_filam.groupby('anio')['indice'].max()
ordenado = pd.merge(df_america, df_filam, on=['anio','indice'],how='inner').dropna().reset_index()
print(ordenado)
df_america = df_filam.groupby('anio')['indice'].min()
ordenado = pd.merge(df_america, df_filam, on=['anio','indice'],how='inner').dropna().reset_index()
print(ordenado)

    index  anio    indice codigo_iso  ranking       pais
0       0  2001     90.30        CUB     99.0       Cuba
1       1  2002     97.83        CUB    125.0       Cuba
2       2  2003  35826.00        ARG     35.0  Argentina
3       3  2004     87.00        CUB    112.0       Cuba
4       4  2005     95.00        CUB    109.0       Cuba
5       5  2006     96.17        CUB    139.0       Cuba
6       6  2007     88.33        CUB    117.0       Cuba
7       7  2008     94.00        CUB    131.0       Cuba
8       8  2009     78.00        CUB    129.0       Cuba
9       9  2012     71.64        CUB    162.0       Cuba
10     10  2013     70.92        CUB    166.0       Cuba
11     11  2014     70.21        CUB    164.0       Cuba
12     12  2015     70.23        CUB    164.0       Cuba
13     42  2017     71.75        CUB    173.0       Cuba
14     43  2018     68.90        CUB    172.0       Cuba
15     44  2019     63.81        CUB    169.0       Cuba
    index  anio  indice codigo_

5. Para cada _país_, muestre el _indice_ máximo que alcanzo por _anio_. Para los datos nulos, rellene con el valor **0**.

> **Hint**: Utilice la función **pd.pivot_table**.



In [255]:
df_lleno = df.fillna(0)
df_acostado = df_lleno.pivot_table(index='pais',columns='anio',values='indice')
df_acostado

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2016,2017,2018,2019
pais,,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,0.0,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,0.0,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,0.0,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,0.0,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,0.0,21.03,22.21,24.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
West Bank y Gaza,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,42.90,42.96,44.68
Yemen,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,0.0,65.80,62.23,61.66
Zambia,26.8,23.25,29.75,23.00,22.50,21.50,15.50,26.75,22.00,27.93,30.89,34.35,35.08,0.0,36.48,35.36,36.38
